# Find exoplanets with missing parameters
* this does the same as FFI so probably don't need, I can delete this later

In [2]:
import os
os.environ['LIGHTKURVE_CACHE'] = "C:/lkcache"

import lightkurve as lk
from astropy.timeseries import BoxLeastSquares
from astropy.io import ascii
from astroquery.mast import Catalogs, Observations
from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

# Finds planets with missing parameters

In [8]:
class MissingParameterFinder:
    """
    Find exoplanet candidates with incomplete parameters that need Batman modeling
    """
    
    def __init__(self):
        self.candidates = None
        self.filtered_candidates = None
        
    def query_toi_catalog(self):
        """
        Query the TESS Objects of Interest (TOI) catalog
        Returns candidates with their known and missing parameters
        """
        print("Querying TOI catalog from ExoFOP-TESS...")
        
        # Download TOI catalog from ExoFOP
        url = "https://exofop.ipac.caltech.edu/tess/download_toi.php?sort=toi&output=csv"
        
        try:
            toi_data = pd.read_csv(url, comment='#')
            print(f"Found {len(toi_data)} TOI candidates")
            
            # Print available columns to see what we're working with
            print("\nAvailable columns in TOI catalog:")
            print(toi_data.columns.tolist())
            
            self.candidates = toi_data
            return toi_data
        except Exception as e:
            print(f"Error downloading TOI catalog: {e}")
            print("Creating sample data instead...")
            return self._create_sample_data()
    
    def _create_sample_data(self):
        """Create sample TOI data for demonstration"""
        sample = pd.DataFrame({
            'TOI': [100.01, 200.01, 300.01, 400.01, 500.01],
            'TIC ID': [12345678, 23456789, 34567890, 45678901, 56789012],
            'Period (days)': [3.5, 12.4, 8.7, 20.1, 5.3],
            'Epoch (BJD)': [2458000.5, 2458100.3, 2458200.1, 2458300.8, 2458400.2],
            'Depth (ppm)': [5000, 2000, 8000, 1500, 3500],
            'Duration (hours)': [2.5, 4.2, 3.1, 5.5, 2.8],
            'Planet Radius (R_Earth)': [2.5, np.nan, 4.1, np.nan, 1.8],
            'a/R_star': [8.5, np.nan, 12.3, np.nan, 7.2],
            'Inclination (deg)': [89.5, np.nan, 88.2, np.nan, 87.5],
            'Eccentricity': [0.0, np.nan, np.nan, np.nan, 0.0],
        })
        self.candidates = sample
        return sample
    
    def identify_missing_parameters(self):
        """
        Analyze which Batman parameters are missing or poorly constrained
        
        Batman REQUIRED parameters:
        1. t0 - Time of transit center (epoch)
        2. per - Orbital period
        3. rp - Planet radius (Rp/R*) - can derive from depth
        4. a - Semi-major axis (a/R*) - often missing
        5. inc - Inclination - often missing or poorly constrained
        6. ecc - Eccentricity - often assumed 0 or missing
        7. w - Argument of periastron - often missing (only matters if ecc > 0)
        8. u - Limb darkening coefficients - need stellar parameters
        
        From TESS we typically GET:
        - Period (well constrained)
        - T0/Epoch (well constrained)
        - Depth (well constrained) → can estimate Rp/R*
        - Duration (reasonably constrained)
        
        From TESS we often CANNOT determine well:
        - a/R* (need stellar parameters OR derive from duration + period + inc)
        - Inclination (need detailed light curve modeling)
        - Eccentricity (need RV data or careful photometric modeling)
        - Argument of periastron (need eccentricity measurement first)
        """
        
        if self.candidates is None:
            print("No candidates loaded. Run query_toi_catalog() first.")
            return None
        
        df = self.candidates.copy()
        
        # Create columns to track what we have and what we need
        required_for_batman = {
            'has_period': 'Period (days)',
            'has_epoch': 'Epoch (BJD)',
            'has_depth': 'Depth (ppm)',
            'has_duration': 'Duration (hours)',
            'has_rp': 'Planet Radius (R_Earth)',
            'has_a_over_rstar': 'a/R_star',
            'has_inclination': 'Inclination (deg)',
            'has_eccentricity': 'Eccentricity'
        }
        
        print("\n" + "="*80)
        print("BATMAN PARAMETER REQUIREMENTS ANALYSIS")
        print("="*80)
        
        print("\nParameters we can GET from TESS light curve:")
        print("  ✓ Period (days) - from BLS or periodogram")
        print("  ✓ T0/Epoch (BJD) - from transit timing")
        print("  ✓ Transit depth (ppm) - from light curve")
        print("  ✓ Transit duration (hours) - from light curve")
        
        print("\nParameters we can ESTIMATE from depth:")
        print("  → Rp/R* (planet-to-star radius ratio) = sqrt(depth)")
        
        print("\nParameters we NEED from modeling or external data:")
        print("  ✗ a/R* (semi-major axis in stellar radii)")
        print("    - Need: Stellar mass/radius OR")
        print("    - Can derive from: Period + Duration + Inclination")
        print("  ✗ Inclination (degrees)")
        print("    - Need: Detailed light curve shape modeling")
        print("    - Affects: Transit depth and duration")
        print("  ✗ Eccentricity")
        print("    - Hard to constrain from photometry alone")
        print("    - Usually need RV data")
        print("  ✗ Argument of periastron (omega)")
        print("    - Only relevant if eccentricity > 0")
        print("  ✗ Limb darkening coefficients")
        print("    - Need: Stellar parameters (Teff, logg, [Fe/H])")
        
        print("\n" + "="*80)
        print("CANDIDATE ANALYSIS")
        print("="*80)
        
        # Check which parameters are missing
        for col_name, col_key in required_for_batman.items():
            if col_key in df.columns:
                n_missing = df[col_key].isna().sum()
                n_total = len(df)
                print(f"{col_key:30s}: {n_total-n_missing}/{n_total} available ({n_missing} missing)")
        
        return df
    
    def filter_candidates_for_modeling(self, 
                                       require_missing_params=False,
                                       tfopwg_disp=None,
                                       max_candidates=100):
        """
        Filter candidates that are good for Batman/MCMC modeling.
        
        Strategy: Since most TOIs have estimated parameters, we look for:
        1. Candidates with basic observables (period, epoch, depth, duration)
        2. Specific TFOPWG dispositions that indicate they need follow-up
        3. Or simply take a sample of all candidates since Batman can refine ANY parameters
        
        Parameters:
        -----------
        require_missing_params : bool
            If True, require some parameters to be missing (likely finds nothing)
        tfopwg_disp : list or None
            Filter by TFOPWG disposition (e.g., ['PC', 'CP', 'APC'])
            PC = Planet Candidate, CP = Confirmed Planet, APC = Ambiguous Planet Candidate
        max_candidates : int
            Maximum number of candidates to return
        """
        
        if self.candidates is None:
            self.query_toi_catalog()
        
        df = self.candidates.copy()
        
        print("\n" + "="*80)
        print("FILTERING CANDIDATES FOR BATMAN MODELING")
        print("="*80)
        
        # Find correct column names (they may vary)
        period_col = None
        epoch_col = None
        depth_col = None
        duration_col = None
        disp_col = None
        
        for col in df.columns:
            col_lower = col.lower()
            if 'period' in col_lower and 'err' not in col_lower:
                period_col = col
            elif 'epoch' in col_lower and 'err' not in col_lower:
                epoch_col = col
            elif 'depth' in col_lower and 'err' not in col_lower:
                depth_col = col
            elif 'duration' in col_lower and 'err' not in col_lower:
                duration_col = col
            elif 'tfopwg' in col_lower or 'disposition' in col_lower:
                disp_col = col
        
        print(f"\nFound columns:")
        print(f"  Period: {period_col}")
        print(f"  Epoch: {epoch_col}")
        print(f"  Depth: {depth_col}")
        print(f"  Duration: {duration_col}")
        print(f"  Disposition: {disp_col}")
        
        # Must have these basic parameters from TESS
        mask = pd.Series([True] * len(df))
        
        if period_col:
            col_mask = df[period_col].notna()
            print(f"\n{period_col}:")
            print(f"  Before: {mask.sum()} candidates")
            mask = mask & col_mask
            print(f"  After: {mask.sum()} candidates")
        
        if epoch_col:
            col_mask = df[epoch_col].notna()
            print(f"\n{epoch_col}:")
            print(f"  Before: {mask.sum()} candidates")
            mask = mask & col_mask
            print(f"  After: {mask.sum()} candidates")
        
        if depth_col:
            col_mask = df[depth_col].notna()
            print(f"\n{depth_col}:")
            print(f"  Before: {mask.sum()} candidates")
            mask = mask & col_mask
            print(f"  After: {mask.sum()} candidates")
        
        # Filter by disposition if requested
        if tfopwg_disp is not None and disp_col is not None:
            disp_mask = df[disp_col].isin(tfopwg_disp)
            print(f"\nFiltering by TFOPWG disposition {tfopwg_disp}:")
            print(f"  Before: {mask.sum()} candidates")
            mask = mask & disp_mask
            print(f"  After: {mask.sum()} candidates")
            
            # Show disposition distribution
            print(f"\nDisposition distribution in filtered set:")
            print(df[mask][disp_col].value_counts())
        
        filtered = df[mask].copy()
        
        # Limit to max_candidates
        if len(filtered) > max_candidates:
            print(f"\nLimiting to first {max_candidates} candidates")
            filtered = filtered.head(max_candidates)
        
        self.filtered_candidates = filtered
        
        print("\n" + "="*80)
        print(f"FINAL RESULT: {len(filtered)} candidates ready for Batman modeling")
        print("="*80)
        print("\nIMPORTANT: Even though these candidates may have estimated parameters,")
        print("Batman/MCMC can REFINE these estimates with detailed light curve modeling!")
        print("Parameters like a/R*, inc, and ecc are often rough estimates that need improvement.")
        
        return filtered
    
    def get_candidate_summary(self, toi_number=None, tic_id=None, index=0):
        """
        Get detailed summary of a specific candidate showing what we have
        and what Batman will need to estimate
        """
        
        if self.filtered_candidates is None or len(self.filtered_candidates) == 0:
            print("No filtered candidates. Run filter_candidates_for_modeling() first")
            return None
        
        # Select candidate
        if toi_number is not None:
            matches = self.filtered_candidates[
                self.filtered_candidates['TOI'] == toi_number
            ]
            if len(matches) == 0:
                print(f"TOI {toi_number} not found in filtered candidates")
                return None
            candidate = matches.iloc[0]
        elif tic_id is not None:
            # Try different possible TIC column names
            tic_cols = [col for col in self.filtered_candidates.columns if 'tic' in col.lower()]
            matches = None
            for col in tic_cols:
                matches = self.filtered_candidates[
                    self.filtered_candidates[col] == tic_id
                ]
                if len(matches) > 0:
                    break
            if matches is None or len(matches) == 0:
                print(f"TIC {tic_id} not found in filtered candidates")
                return None
            candidate = matches.iloc[0]
        else:
            candidate = self.filtered_candidates.iloc[index]
        
        print("\n" + "="*80)
        print("CANDIDATE PARAMETER SUMMARY FOR BATMAN MODELING")
        print("="*80)
        
        # Print TOI and TIC
        for col in candidate.index:
            if col.lower() == 'toi':
                print(f"\nTOI: {candidate[col]}")
            elif 'tic' in col.lower() and 'id' in col.lower():
                print(f"TIC ID: {candidate[col]}")
        
        print("\n--- PARAMETERS WE HAVE (from TESS observations) ---")
        
        # Find the actual column names
        for col in candidate.index:
            col_lower = col.lower()
            
            # Period
            if 'period' in col_lower and 'err' not in col_lower and pd.notna(candidate[col]):
                print(f"  ✓ {col:40s} = {candidate[col]:12.6f}  (Batman: per)")
            
            # Epoch/T0
            elif 'epoch' in col_lower and 'err' not in col_lower and pd.notna(candidate[col]):
                print(f"  ✓ {col:40s} = {candidate[col]:12.6f}  (Batman: t0)")
            
            # Depth
            elif 'depth' in col_lower and 'err' not in col_lower and pd.notna(candidate[col]):
                depth_val = candidate[col]
                print(f"  ✓ {col:40s} = {depth_val:12.6f} ppm")
                rp_over_rs = np.sqrt(depth_val / 1e6)
                print(f"  → Rp/R* (derived from depth)            = {rp_over_rs:12.6f}  (Batman: rp)")
            
            # Duration
            elif 'duration' in col_lower and 'err' not in col_lower and pd.notna(candidate[col]):
                print(f"  ✓ {col:40s} = {candidate[col]:12.6f}  (for constraints)")
        
        print("\n--- PARAMETERS TO REFINE/ESTIMATE (using Batman/MCMC) ---")
        
        # Look for a/R*, inclination, eccentricity
        for col in candidate.index:
            col_lower = col.lower()
            
            # a/R*
            if ('a/r' in col_lower or 'semi' in col_lower) and 'err' not in col_lower:
                if pd.notna(candidate[col]):
                    print(f"  ~ {col:40s} = {candidate[col]:12.6f}  (Batman: a - can refine)")
                else:
                    print(f"  ✗ {col:40s} = MISSING (Batman: a - estimate)")
            
            # Inclination
            elif 'inc' in col_lower and 'err' not in col_lower and 'planet' not in col_lower:
                if pd.notna(candidate[col]):
                    print(f"  ~ {col:40s} = {candidate[col]:12.6f}° (Batman: inc - can refine)")
                else:
                    print(f"  ✗ {col:40s} = MISSING (Batman: inc - estimate)")
            
            # Eccentricity
            elif 'ecc' in col_lower and 'err' not in col_lower:
                if pd.notna(candidate[col]):
                    print(f"  ~ {col:40s} = {candidate[col]:12.6f}  (Batman: ecc - can refine)")
                else:
                    print(f"  ✗ {col:40s} = MISSING (Batman: ecc - estimate/fix to 0)")
        
        print("\n--- ADDITIONAL BATMAN REQUIREMENTS ---")
        print("  ✗ w (argument of periastron)       = Usually fix to 90° for circular orbits")
        print("  ✗ u (limb darkening coefficients)  = Need stellar Teff, logg, [Fe/H]")
        print("                                       Lookup from tables or TIC catalog")
        
        print("\n--- WHAT BATMAN/MCMC WILL DO ---")
        print("  Even if parameters like a/R*, inc, ecc exist in the catalog,")
        print("  they are often ROUGH ESTIMATES from:")
        print("    - Automated pipelines")
        print("    - Assumptions (e.g., circular orbits)")
        print("    - Limited light curve fitting")
        print("  ")
        print("  Your Batman/MCMC analysis will REFINE these by:")
        print("    1. Fitting the detailed light curve shape")
        print("    2. Properly accounting for limb darkening")
        print("    3. Exploring parameter correlations")
        print("    4. Providing robust uncertainty estimates")
        
        print("\n--- VALUES YOU'LL PROVIDE TO BATMAN ---")
        print("  From this candidate, extract:")
        
        # Show what to extract
        period_col = [c for c in candidate.index if 'period' in c.lower() and 'err' not in c.lower()]
        epoch_col = [c for c in candidate.index if 'epoch' in c.lower() and 'err' not in c.lower()]
        depth_col = [c for c in candidate.index if 'depth' in c.lower() and 'err' not in c.lower()]
        
        if period_col:
            print(f"    per = {candidate[period_col[0]]:.6f}  # days")
        if epoch_col:
            print(f"    t0 = {candidate[epoch_col[0]]:.6f}  # BJD")
        if depth_col:
            rp = np.sqrt(candidate[depth_col[0]] / 1e6)
            print(f"    rp = {rp:.6f}  # Rp/R* (from depth)")
        print(f"    a = ???  # To estimate/refine with MCMC")
        print(f"    inc = ???  # To estimate/refine with MCMC")
        print(f"    ecc = 0.0  # Start with circular, refine if needed")
        print(f"    w = 90.0  # degrees")
        print(f"    u = [u1, u2]  # Get from stellar parameters")
        
        print("="*80 + "\n")
        
        return candidate
    
    def export_candidates_for_batman(self, filename='candidates_for_batman.csv'):
        """
        Export filtered candidates with clear indication of what parameters
        are available and what need to be estimated
        """
        
        if self.filtered_candidates is None:
            print("No filtered candidates. Run filter_candidates_for_modeling() first")
            return
        
        df = self.filtered_candidates.copy()
        
        # Add derived parameter
        if 'Depth (ppm)' in df.columns:
            df['Rp/R* (derived)'] = np.sqrt(df['Depth (ppm)'] / 1e6)
        
        # Add status columns
        df['needs_a_over_rstar'] = df['a/R_star'].isna() if 'a/R_star' in df.columns else True
        df['needs_inclination'] = df['Inclination (deg)'].isna() if 'Inclination (deg)' in df.columns else True
        df['needs_eccentricity'] = df['Eccentricity'].isna() if 'Eccentricity' in df.columns else True
        
        df.to_csv(filename, index=False)
        print(f"\nExported {len(df)} candidates to {filename}")
        print("\nColumns include:")
        print("  - Observable parameters: Period, Epoch, Depth, Duration")
        print("  - Derived parameters: Rp/R*")
        print("  - Parameters to estimate: a/R*, Inclination, Eccentricity")
        print("  - Status flags: needs_a_over_rstar, needs_inclination, needs_eccentricity")
        
        return filename
    
    def download_light_curve_for_candidate(self, toi_number=None, tic_id=None, 
                                           index=0, sector=None):
        """
        Download TESS light curve for a specific candidate
        Returns the light curve object ready for Batman modeling
        """
        
        # Get candidate info
        if self.filtered_candidates is None or len(self.filtered_candidates) == 0:
            print("No filtered candidates available")
            return None
        
        if toi_number is not None:
            candidate = self.filtered_candidates[
                self.filtered_candidates['TOI'] == toi_number
            ].iloc[0]
        elif tic_id is not None:
            candidate = self.filtered_candidates[
                self.filtered_candidates['TIC ID'] == tic_id
            ].iloc[0]
        else:
            candidate = self.filtered_candidates.iloc[index]
        
        # Get TIC ID
        target_tic = candidate['TIC ID'] if 'TIC ID' in candidate.index else None
        target_toi = candidate['TOI'] if 'TOI' in candidate.index else None
        
        if target_tic is None and target_toi is None:
            print("No TIC ID or TOI found for this candidate")
            return None
        
        print(f"\nDownloading TESS data for TIC {target_tic} (TOI {target_toi})")
        
        # Search for light curve
        search_result = lk.search_lightcurve(
            f'TIC {target_tic}',
            mission='TESS',
            sector=sector
        )
        
        if len(search_result) == 0:
            print("No TESS data found for this target")
            return None
        
        print(f"Found {len(search_result)} light curve(s)")
        print(search_result)
        
        # Download
        lc_collection = search_result.download_all()
        lc = lc_collection.stitch()
        
        # Clean
        lc = lc.remove_nans().normalize().remove_outliers(sigma=5)
        
        print(f"Light curve ready: {len(lc.time)} data points")
        print(f"Time range: {lc.time.value.min():.2f} to {lc.time.value.max():.2f} BTJD")
        
        print("\n--- READY FOR BATMAN MODELING ---")
        print("Use these values to initialize Batman:")
        print(f"  time array: lc.time.value  ({len(lc.time)} points)")
        print(f"  flux array: lc.flux.value")
        if 'Period (days)' in candidate.index:
            print(f"  params.per = {candidate['Period (days)']}")
        if 'Epoch (BJD)' in candidate.index:
            print(f"  params.t0 = {candidate['Epoch (BJD)']}")
        if 'Depth (ppm)' in candidate.index:
            depth = candidate['Depth (ppm)']
            rp = np.sqrt(depth / 1e6)
            print(f"  params.rp = {rp:.6f}  (from depth = {depth:.1f} ppm)")
        
        return lc


# Example usage
if __name__ == "__main__":
    
    # Initialize finder
    finder = MissingParameterFinder()
    
    # Step 1: Query TOI catalog
    print("\n" + "="*80)
    print("STEP 1: QUERYING TOI CATALOG")
    print("="*80)
    toi_data = finder.query_toi_catalog()
    
    # Step 2: Analyze what parameters are available
    print("\n" + "="*80)
    print("STEP 2: ANALYZING AVAILABLE PARAMETERS")
    print("="*80)
    finder.identify_missing_parameters()
    
    # Step 3: Filter for candidates good for Batman modeling
    print("\n" + "="*80)
    print("STEP 3: FILTERING CANDIDATES")
    print("="*80)
    
    # Option 1: Get all planet candidates (good for refinement)
    candidates = finder.filter_candidates_for_modeling(
        require_missing_params=False,
        tfopwg_disp=['PC', 'CP'],  # Planet Candidate or Confirmed Planet
        max_candidates=50  # Limit to first 50 for this example
    )
    
    # Option 2: If you want ALL candidates regardless of disposition
    # candidates = finder.filter_candidates_for_modeling(
    #     require_missing_params=False,
    #     tfopwg_disp=None,
    #     max_candidates=100
    # )
    
    # Step 4: Get detailed summary of first candidate
    print("\n" + "="*80)
    print("STEP 4: EXAMPLE CANDIDATE SUMMARY")
    print("="*80)
    if len(candidates) > 0:
        candidate_info = finder.get_candidate_summary(index=0)
    
    # Step 5: Export candidates
    print("\n" + "="*80)
    print("STEP 5: EXPORTING CANDIDATES")
    print("="*80)
    finder.export_candidates_for_batman('toi_candidates_for_batman.csv')
    
    # Step 6: Example of downloading light curve for modeling
    print("\n" + "="*80)
    print("STEP 6: DOWNLOADING LIGHT CURVE (EXAMPLE)")
    print("="*80)
    print("(Uncomment the line below to download data)")
    # lc = finder.download_light_curve_for_candidate(index=0)


STEP 1: QUERYING TOI CATALOG
Querying TOI catalog from ExoFOP-TESS...
Found 7821 TOI candidates

Available columns in TOI catalog:
['TIC ID', 'TOI', 'Previous CTOI', 'Master', 'SG1A', 'SG1B', 'SG2', 'SG3', 'SG4', 'SG5', 'ESM', 'TSM', 'Predicted Mass (M_Earth)', 'Predicted RV Semi-amplitude (m/s)', 'Time Series Observations', 'Spectroscopy Observations', 'Imaging Observations', 'TESS Disposition', 'TFOPWG Disposition', 'TESS Mag', 'TESS Mag err', 'Planet Name', 'Pipeline Signal ID', 'Source', 'Detection', 'RA', 'Dec', 'PM RA (mas/yr)', 'PM RA err (mas/yr)', 'PM Dec (mas/yr)', 'PM Dec err (mas/yr)', 'Epoch (BJD)', 'Epoch (BJD) err', 'Period (days)', 'Period (days) err', 'Duration (hours)', 'Duration (hours) err', 'Depth (mmag)', 'Depth (mmag) err', 'Depth (ppm)', 'Depth (ppm) err', 'Planet Radius (R_Earth)', 'Planet Radius (R_Earth) err', 'Planet Insolation (Earth Flux)', 'Planet Equil Temp (K)', 'Planet SNR', 'Stellar Distance (pc)', 'Stellar Distance (pc) err', 'Stellar Eff Temp (K)',

# Example

In [10]:
if __name__ == "__main__":
    
    # Initialize finder
    finder = MissingParameterFinder()
    
    # Step 1: Query TOI catalog
    print("\n" + "="*80)
    print("STEP 1: QUERYING TOI CATALOG")
    print("="*80)
    toi_data = finder.query_toi_catalog()
    
    # Step 2: Analyze what parameters are available
    print("\n" + "="*80)
    print("STEP 2: ANALYZING AVAILABLE PARAMETERS")
    print("="*80)
    finder.identify_missing_parameters()
    
    # Step 3: Filter for candidates good for Batman modeling
    print("\n" + "="*80)
    print("STEP 3: FILTERING CANDIDATES")
    print("="*80)
    candidates = finder.filter_candidates_for_modeling(
        require_light_curve=True,
        require_missing_params=True
    )
    
    # Step 4: Get detailed summary of first candidate
    print("\n" + "="*80)
    print("STEP 4: EXAMPLE CANDIDATE SUMMARY")
    print("="*80)
    if len(candidates) > 0:
        candidate_info = finder.get_candidate_summary(index=0)
    
    # Step 5: Export candidates
    print("\n" + "="*80)
    print("STEP 5: EXPORTING CANDIDATES")
    print("="*80)
    finder.export_candidates_for_batman('toi_candidates_for_batman.csv')
    
    # Step 6: Example of downloading light curve for modeling
    print("\n" + "="*80)
    print("STEP 6: DOWNLOADING LIGHT CURVE (EXAMPLE)")
    print("="*80)
    print("(Uncomment the line below to download data)")
    # lc = finder.download_light_curve_for_candidate(index=0)


STEP 1: QUERYING TOI CATALOG
Querying TOI catalog from ExoFOP-TESS...
Found 7821 TOI candidates

Available columns in TOI catalog:
['TIC ID', 'TOI', 'Previous CTOI', 'Master', 'SG1A', 'SG1B', 'SG2', 'SG3', 'SG4', 'SG5', 'ESM', 'TSM', 'Predicted Mass (M_Earth)', 'Predicted RV Semi-amplitude (m/s)', 'Time Series Observations', 'Spectroscopy Observations', 'Imaging Observations', 'TESS Disposition', 'TFOPWG Disposition', 'TESS Mag', 'TESS Mag err', 'Planet Name', 'Pipeline Signal ID', 'Source', 'Detection', 'RA', 'Dec', 'PM RA (mas/yr)', 'PM RA err (mas/yr)', 'PM Dec (mas/yr)', 'PM Dec err (mas/yr)', 'Epoch (BJD)', 'Epoch (BJD) err', 'Period (days)', 'Period (days) err', 'Duration (hours)', 'Duration (hours) err', 'Depth (mmag)', 'Depth (mmag) err', 'Depth (ppm)', 'Depth (ppm) err', 'Planet Radius (R_Earth)', 'Planet Radius (R_Earth) err', 'Planet Insolation (Earth Flux)', 'Planet Equil Temp (K)', 'Planet SNR', 'Stellar Distance (pc)', 'Stellar Distance (pc) err', 'Stellar Eff Temp (K)',

TypeError: MissingParameterFinder.filter_candidates_for_modeling() got an unexpected keyword argument 'require_light_curve'